In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
print(os.environ["MODEL_ID"])

meta-llama/Meta-Llama-3-8B-Instruct


In [2]:
from datetime import datetime, timedelta

import faiss
from langchain.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_community.embeddings import HuggingFaceEmbeddings

decay_rate이 낮을 수록 오래 기억하고 높을 수록 금방 잊혀진다는 의미

In [5]:
# 임베딩 모델을 정의합니다.
embeddings_model = HuggingFaceEmbeddings()
# 벡터 저장소를 빈 상태로 초기화합니다.
embedding_size = 768
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})
# 시간 가중치가 적용된 벡터 저장소 검색기를 초기화합니다.
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore, decay_rate=0.0000000000000000000000001, k=1
)

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
yesterday = datetime.now() - timedelta(days=1)  # 어제 날짜를 계산합니다.
retriever.add_documents(
    # "hello world" 내용의 문서를 추가하고, 메타데이터에 어제 날짜를 설정합니다.
    [Document(page_content="hello world", metadata={
              "last_accessed_at": yesterday})]
)
# "hello foo" 내용의 문서를 추가합니다.
retriever.add_documents([Document(page_content="hello foo")])

['7c3971f4-f29f-4b98-91a0-d597951ce2d9']

In [7]:
# "Hello World"가 가장 먼저 반환되는 이유는 가장 두드러지기 때문이며, 감쇠율이 0에 가깝기 때문에 여전히 최신 상태를 유지하고 있음을 의미합니다.
retriever.get_relevant_documents("hello world")

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[Document(page_content='hello world', metadata={'last_accessed_at': datetime.datetime(2024, 6, 14, 0, 27, 18, 372175), 'created_at': datetime.datetime(2024, 6, 14, 0, 26, 52, 18763), 'buffer_idx': 0})]

In [8]:
# 임베딩 모델을 정의합니다.
embeddings_model = HuggingFaceEmbeddings()
# 벡터 저장소를 빈 상태로 초기화합니다.
embedding_size = 768
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(embeddings_model, index, InMemoryDocstore({}), {})
# 시간 가중치가 적용된 벡터 저장소 검색기를 초기화합니다.
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore, decay_rate=0.999, k=1
)

/home/dudaji/anaconda3/envs/llm-study/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [9]:
yesterday = datetime.now() - timedelta(days=1)  # 어제 날짜를 계산합니다.
retriever.add_documents(
    # "hello world" 내용의 문서를 추가하고, 메타데이터에 어제 날짜를 설정합니다.
    [Document(page_content="hello world", metadata={
              "last_accessed_at": yesterday})]
)
# "hello foo" 내용의 문서를 추가합니다.
retriever.add_documents([Document(page_content="hello foo")])

['792b5a96-365e-4b0b-818f-0147ac1c94d0']

In [10]:
# "hello world"와 가장 관련된 문서를 검색합니다.
# "Hello Foo"가 먼저 반환되는 이유는 "hello world"가 대부분 잊혀졌기 때문입니다.
retriever.get_relevant_documents("hello world")

[Document(page_content='hello foo', metadata={'last_accessed_at': datetime.datetime(2024, 6, 14, 0, 28, 29, 538832), 'created_at': datetime.datetime(2024, 6, 14, 0, 27, 56, 702558), 'buffer_idx': 1})]